In [31]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# pandas
import pandas as pd

In [32]:
def get_youtube_main_video_all_data(url):
    
    # webdriver 초기화
    driver_path ='./chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    time.sleep(5)
    
    thumbnail_list = [] # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = [] # 썸네일 제목을 저장하는 리스트
    overlay_time_list = [] # 동영상 재생시간들을 저장하는 리스트
    channel_profile_list = [] # 채널 프로필사진 주소 저장용 리스트
    channel_name_list = [] # 채널 이름 저장용 리스트
    hits_str_list = [] # 조회수 문자열 저장용 리스트
    update_date_list = [] # 업로드 시간 저장용 리스트
    
    # 첫 동영상 시작 시 시작번호(index = 1)
    index = 1
    # 한 번 스크롤하고 멈춤 시간 설정
    SCROLL_PAUSE_TIME = 2
    
    # num_of_pagedowns = 10
    
    while True:
        try:
            thumbnail_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
            overlay_time_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span'
            channel_profile_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/a/yt-img-shadow/img'
            channel_name_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
            hits_str_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[1]'
            update_date_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[2]'

            # 이미지, 채널 프로필, 오버레이 타임이 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            thumb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, thumbnail_xpath)))
            overlay = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, overlay_time_xpath)))
            ch_profile = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, channel_profile_xpath)))

            if (thumb or overlay or ch_profile) is None:
                print("img or overlay or ch_profile is not loaded.")

            # 동영상 목록을 추가 불러오기 위해 스크롤 내림(pass)
            # index % 8 일 경우 스크롤을 내림(8개씩 다 볼 경우) -> page_down key로 내린다
            if index % 8 == 0:
                # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                
            # 썸네일 주소를 리스트에 저장
            thumbnail = driver.find_element_by_xpath(thumbnail_xpath)
            thumbnail_url = thumbnail.get_attribute('src')
            thumbnail_list.append(thumbnail_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # 동영상 재생시간을 리스트에 저장
            overlay_time = driver.find_element_by_xpath(overlay_time_xpath)
            overlay_time_list.append(overlay_time.text)

            # 채널 프로필 사진 링크를 리스트에 저장
            channel_profile = driver.find_element_by_xpath(channel_profile_xpath)
            channel_profile_url = channel_profile.get_attribute('src')
            channel_profile_list.append(channel_profile_url)

            # 채널이름을 리스트에 저장
            channel_name = driver.find_element_by_xpath(channel_name_xpath)
            channel_name_list.append(channel_name.text)

            # 조회수 문자열을 리스트에 저장
            hits_str = driver.find_element_by_xpath(hits_str_xpath)
            hits_str_list.append(hits_str.text)

            # 업데이트 시간을 리스트에 저장
            update_date = driver.find_element_by_xpath(update_date_xpath)
            update_date_list.append(update_date.text)

            # 리스트에 들어간 데이터 중간 확인
            print('index_no: ', index)
            print('title_text:', title.text, ' img_url: ', thumbnail_url)
            print('overlay_time: ', overlay_time.text, ' channel_profile: ', channel_profile_url)
            print('channel_name: ', channel_name.text, ' hits_str: ', hits_str.text, ' update_date: ', update_date.text) 
            
            index += 1 # 다음 영상데이터로 넘어간다
        
        except Exception as e:
            print()
            print("Exception: ", e)
            break
    
    driver.close()
    
    # pandas DataFrame 형태로 데이터를 테이블에 담아 리턴
    for thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date in zip(thumbnail_list, title_list, overlay_time_list, channel_profile_list, channel_name_list,
        hits_str_list, update_date_list):
        print(thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date)
    
    # making dataframe
    dic = {'thumbnail': thumbnail_list, 'title': title_list, 'overlay_time': overlay_time_list, 'channel_profile': channel_profile_list,
          'channel_name': channel_name_list, 'hits_str': hits_str_list, 'update_date': update_date_list}
    
    df = pd.DataFrame(dic)
    
    return df

In [33]:
test_url = 'https://www.youtube.com/'
get_youtube_main_video_all_data(test_url)

index_no:  1
title_text: 🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 11월 10일 2주차🔥KPOP Chart🔥 최신 인기가요 노래 모음!!🔥🔥  img_url:  https://i.ytimg.com/vi/EUW6MxGg64A/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAAbJjjUZe0QAmdpdP8vk_7wDlk9Q
overlay_time:  5:51:31  channel_profile:  https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5tiwKsokaNQF-zLeVshujwQ=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  멜론둥이  hits_str:  조회수 8만회  update_date:  13시간 전
index_no:  2
title_text: [무한도전] ((무한상사)) 할말하않...💢 세상 극한직업 유부장과 노답(?) 직원들 (^_−)☆  img_url:  https://i.ytimg.com/vi/tYM4oISacwY/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD4d1VmexCqCwIfaJUmXSuitppmDg
overlay_time:  8:30  channel_profile:  https://yt3.ggpht.com/a-/AOh14Gg0jft0lRv6zvU8pGCGN-VpIl2BzGTIWDRs3w=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  MBCentertainment  hits_str:  조회수 659만회  update_date:  7년 전
index_no:  3
title_text: [#스트리트푸드파이터] 고소한 고추기름 + 고소한 건두부 = 꼬소 뽝💥 공깃밥 뚝딱 훔.쳐.가.요 -괴도 건두부 무침- | #다시보는스푸파 #Diggle  img_url:  https

index_no:  21
title_text: [짤툰 오리지널] 금연  img_url:  https://i.ytimg.com/vi/S2Qysqt-7jE/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAdGzm8SNgIOOmm6vo8X6eAw5RCTw
overlay_time:  3:55  channel_profile:  https://yt3.ggpht.com/a-/AOh14Gj07QwC3x3DsbiJTszf4u_ZQLNJD6oaLNuKZw=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  짤툰  hits_str:  조회수 269만회  update_date:  3주 전
index_no:  22
title_text: [#랜덤박스] 쨍쨍한 날 웬 비야? 여우가 시집가서요☔️ 하객으로 가서 신부랑 맞짱뜨는 구미호 이동욱 ㄷㄷ 동양설화 맛집 구미호뎐! | #구미호뎐 #Diggle  img_url:  https://i.ytimg.com/vi/aGJUM1MeY0s/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCgPoG4z2C9X0_UaJQnnLJPN2qb0A
overlay_time:  9:05  channel_profile:  https://yt3.ggpht.com/a-/AOh14GjRI0f_IlwSr7bHO8jtYjAWOtj0uHwK4YMpng=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  tvN D ENT  hits_str:  조회수 182만회  update_date:  3주 전
index_no:  23
title_text: 50년 된 4평 원룸 꾸미기  img_url:  https://i.ytimg.com/vi/8NhdWrtTR1w/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLALde

index_no:  41
title_text: 🎤언제 들어도 소름 돋는 이하이(Lee Hi)의 ′Rose🌹♪ 〈비긴어게인 코리아(beginagainkorea)〉 4회  img_url:  https://i.ytimg.com/vi/cq-A2RJC7fg/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDHb-3KP7p6ZQmXuGtqYcf3_kQk0Q
overlay_time:  3:21  channel_profile:  https://yt3.ggpht.com/a-/AOh14Gh6YysJtGL0PhvzJmZz-WqQuxwqOWMK4NjSJQ=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  Beginagain 비긴어게인  hits_str:  조회수 1028만회  update_date:  4개월 전
index_no:  42
title_text: [ENG] SMTM9 [2회] 마미손의 삼고초려?! 독보적 음악천재, 원슈타인 @2차 예선 EP.2 201023  img_url:  https://i.ytimg.com/vi/DWhqxUOWTts/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDJt8pC1qZFeEmmsCncepi7eCejTA
overlay_time:  3:38  channel_profile:  https://yt3.ggpht.com/a-/AOh14GjArOls-JxqrLd-eFRKx2_WTdRjh-D2b0NDUw=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  Mnet Official  hits_str:  조회수 213만회  update_date:  2주 전
index_no:  43
title_text: “거짓으로 내 이름 팔지 마!” 백종원, 백반집 사장에 대폭발 분노 @백종원의 골목식당 78회 20190807  img_url:  https://i.ytim

index_no:  60
title_text: 나에게 주는 것 없이 좋은 사람 #멋쟁이희극인박지선｜크랩  img_url:  https://i.ytimg.com/vi/4R2zVnPughw/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLArX7FG0bxbcN570laJ7KfnySOV0g
overlay_time:  4:47  channel_profile:  https://yt3.ggpht.com/a-/AOh14Ggxn8C0NiIgZuf8MxviiKptwCSUd8BW7R_lEg=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  크랩 KLAB  hits_str:  조회수 122만회  update_date:  6일 전
index_no:  61
title_text: 마음을 편하게 해주는 아름다운 클래식 피아노 | Kiss The Sky 12시간 브금  img_url:  https://i.ytimg.com/vi/bnWweoGlEZs/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD52O6MLhwP_LPydn3zk2HnMMYs_g
overlay_time:  11:55:01  channel_profile:  https://yt3.ggpht.com/a-/AOh14GhmyFTFkCXLNrhebuOTgvSaz_yFnFrkvTxiMA=s68-c-k-c0x00ffffff-no-rj-mo
channel_name:  힐링음악 - Healing Music  hits_str:  조회수 153만회  update_date:  1년 전
index_no:  62
title_text: 요리하는 남자가 처가댁에 놀러가면 벌어지는 일  img_url:  https://i.ytimg.com/vi/5GzsHHR4qds/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn

,thumbnail,title,overlay_time,channel_profile,channel_name,hits_str,update_date
0,https://i.ytimg.com/vi/EUW6MxGg64A/hq720.jpg?s...,🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 11월 10일 2주차🔥KPOP ...,5:51:31,https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5t...,멜론둥이,조회수 8만회,13시간 전
1,https://i.ytimg.com/vi/tYM4oISacwY/hq720.jpg?s...,[무한도전] ((무한상사)) 할말하않...💢 세상 극한직업 유부장과 노답(?) 직원...,8:30,https://yt3.ggpht.com/a-/AOh14Gg0jft0lRv6zvU8p...,MBCentertainment,조회수 659만회,7년 전
2,https://i.ytimg.com/vi/OsSziYSdaPY/hq720.jpg?s...,[#스트리트푸드파이터] 고소한 고추기름 + 고소한 건두부 = 꼬소 뽝💥 공깃밥 뚝딱...,8:53,https://yt3.ggpht.com/a-/AOh14GjRI0f_IlwSr7bHO...,tvN D ENT,조회수 90만회,2개월 전
3,https://i.ytimg.com/vi/6T5Q_eUkdFk/hq720.jpg?s...,"無廣告音樂 , 作業用BGM,閱讀和學習音樂,集中的音樂,放鬆音樂,適合放鬆 療癒 舒眠 減...",3:28:32,https://yt3.ggpht.com/a-/AOh14GjZUiXtORz-vxdck...,最佳声音背景音乐 - Relaxing Music,조회수 5.2천회,2주 전
4,https://i.ytimg.com/vi/Fyd1BiP6ipg/hq720.jpg?s...,"[어디선가 들어본 노래 Playlist] 제목은 모르더라도, 한때 지겹도록 들었던",2:23:30,https://yt3.ggpht.com/a-/AOh14Gjq5YofumMoqhCjq...,하루의 안식처_HAAN,조회수 321만회,5개월 전
...,...,...,...,...,...,...,...
59,https://i.ytimg.com/vi/4R2zVnPughw/hq720.jpg?s...,나에게 주는 것 없이 좋은 사람 #멋쟁이희극인박지선｜크랩,4:47,https://yt3.ggpht.com/a-/AOh14Ggxn8C0NiIgZuf8M...,크랩 KLAB,조회수 122만회,6일 전
60,https://i.ytimg.com/vi/bnWweoGlEZs/hq720.jpg?s...,마음을 편하게 해주는 아름다운 클래식 피아노 | Kiss The Sky 12시간 브금,11:55:01,https://yt3.ggpht.com/a-/AOh14GhmyFTFkCXLNrheb...,힐링음악 - Healing Music,조회수 153만회,1년 전
61,https://i.ytimg.com/vi/5GzsHHR4qds/hq720.jpg?s...,요리하는 남자가 처가댁에 놀러가면 벌어지는 일,13:51,https://yt3.ggpht.com/a-/AOh14GjJ_nPvOLq1B6RvU...,취미로 요리하는 남자 Yonam,조회수 121만회,1주 전
62,https://i.ytimg.com/vi/P6srEFTxk2g/hqdefault.j...,"[무한도전] 20초컷으로 가족 소환하던,,그때,,그 시절,,,★ 명수는 12살! 준...",21:21,https://yt3.ggpht.com/a-/AOh14Gg0jft0lRv6zvU8p...,MBCentertainment,조회수 915만회,8년 전
